In [1]:
import pandas as pd
import numpy as np
import re

## We want to unify the MovieLens movieId and IMDBid for our database

### Load Completed IMDB Blockbuster movie dataset from 1_Resolve_IMDB_Disney_Movie_Title

In [2]:
#Load the complete IMDB dataset
complete_df = pd.read_csv('output/complete_Disney_movie.csv')
complete_df['Year'] = complete_df['Year'].str.split('–', 1).str[0]
complete_df['Genre'] = complete_df['Genre'].str.replace(',', '|')
complete_df['Genre'] = complete_df['Genre'].str.replace(' ', '')

complete_df['Director'] = complete_df['Director'].str.replace(',', '|')
#complete_df['Director'] = complete_df['Director'].str.replace(' ', '')

complete_df['Writer'] = complete_df['Writer'].str.replace(',', '|')
#complete_df['Writer'] = complete_df['Writer'].str.replace(' ', '')

complete_df['Actors'] = complete_df['Actors'].str.replace(',', '|')
#complete_df['Actors'] = complete_df['Actors'].str.replace(' ', '')

complete_df['Language'] = complete_df['Language'].str.replace(',', '|')
complete_df['Language'] = complete_df['Language'].str.replace(' ', '')

complete_df['Country'] = complete_df['Country'].str.replace(',', '|')
#complete_df['Country'] = complete_df['Country'].str.replace(' ', '')


#complete_df['imdbVotes'] = complete_df['imdbVotes'].str.replace(',', '')

complete_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,Country,Awards,Metascore,IMDBRating,imdbVotes,imdbID,total_gross,inflation_adjusted_gross
0,101 Dalmatians,1996,G,27 Nov 1996,103.0,Adventure|Comedy|Crime,Stephen Herek,Dodie Smith| John Hughes,Glenn Close| Jeff Daniels| Joely Richardson,English|Spanish,United States| United Kingdom,Nominated for 1 BAFTA Film Award3 wins & 10 no...,49.0,5.7,109762,tt0115433,153000000.0,1.386569e+09
1,101 Dalmatians,1996,G,27 Nov 1996,103.0,Adventure|Comedy|Crime,Stephen Herek,Dodie Smith| John Hughes,Glenn Close| Jeff Daniels| Joely Richardson,English|Spanish,United States| United Kingdom,Nominated for 1 BAFTA Film Award3 wins & 10 no...,49.0,5.7,109762,tt0115433,136189294.0,2.352021e+08
2,102 Dalmatians,2000,G,22 Nov 2000,100.0,Adventure|Comedy|Family,Kevin Lima,Dodie Smith| Kristen Buckley| Brian Regan,Glenn Close| Gérard Depardieu| Ioan Gruffudd,English,United States| United Kingdom,Nominated for 1 Oscar. 1 win & 8 nominations t...,35.0,4.9,37045,tt0211181,66941559.0,1.053377e+08
3,1492: Conquest of Paradise,1992,PG-13,09 Oct 1992,154.0,Adventure|Biography|Drama,Ridley Scott,Rose Bosch,Gérard Depardieu| Armand Assante| Sigourney We...,English|Spanish|Latin|Arabic,United Kingdom| France| Spain,2 nominations,47.0,6.4,29990,tt0103594,7099531.0,1.371176e+07
4,"20,000 Leagues Under the Sea",1954,G,20 Jul 1955,127.0,Adventure|Drama|Family,Richard Fleischer,Earl Felton| Jules Verne,Kirk Douglas| James Mason| Paul Lukas,English,United States,Won 2 Oscars. 5 wins & 3 nominations total,83.0,7.2,33152,tt0046672,28200000.0,2.840652e+08


In [3]:
complete_df['Year'] = complete_df['Year'].astype(int)

### Load MovieLens movies dataset

In [4]:
#load Movie Lens movies dataset
ml_df = pd.read_csv('input/movies.csv')
ml_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
#ml_df['title'] = ml_df['title'].str.replace('.','')
ml_df['movie_title'] = ml_df['title'].str.split(' \(', 1).str[0]
ml_df['movie_title'] = ml_df['movie_title'].str.replace('.','')
ml_df['year'] = ml_df['title'].str.extract('(\(\d{4}\)$)')
ml_df['year'] = ml_df['year'].str.replace('\)','')
ml_df['year'] = ml_df['year'].str.replace('\(','')

In [6]:
#Can't parse year to int as there are some null values
ml_df[ml_df['year'].isnull()]

,movieId,title,genres,movie_title,year
8463,25936,"Babe Ruth Story, The (1948)",Drama,"Babe Ruth Story, The",NaN
8602,26137,"Heroes of Telemark, The (1965)",Action|Drama|War,"Heroes of Telemark, The",NaN
8910,26628,Jack's Back (1988),Crime|Horror|Mystery|Thriller,Jack's Back,NaN
9120,27189,After the Rain (Ame agaru) (1999),Action|Drama,After the Rain,NaN
9763,32497,Love Letter (1995),Drama|Romance,Love Letter,NaN
...,...,...,...,...,...
62071,207714,Tales of Found Footage,(no genres listed),Tales of Found Footage,NaN
62104,207884,Enduring Destiny,(no genres listed),Enduring Destiny,NaN
62285,208597,Punk the Capital: Building a Sound Movement,Documentary,Punk the Capital: Building a Sound Movement,NaN
62326,208763,Yosemite: The Fate of Heaven,(no genres listed),Yosemite: The Fate of Heaven,NaN


In [7]:
ml_df[ml_df.movie_title.str.startswith('101 Dalmatians')]

,movieId,title,genres,movie_title,year
1331,1367,101 Dalmatians (1996),Adventure|Children|Comedy,101 Dalmatians,1996
1996,2085,101 Dalmatians (One Hundred and One Dalmatians...,Adventure|Animation|Children,101 Dalmatians,1961
24273,121099,101 Dalmatians II: Patch's London Adventure (2...,Animation|Children,101 Dalmatians II: Patch's London Adventure,2003


In [8]:
#Fill Null year with -1
ml_df['year'].fillna("-1", inplace = True)
#Parse year as int
ml_df['year'] = ml_df['year'].astype(int)

In [9]:
#genres na?
ml_df.loc[ml_df['genres'].isna()]

,movieId,title,genres,movie_title,year


In [10]:
#Merge the IMDB complete dataset and MovieLens dataset on movie_title & year ONLY
merged_disney_df =pd.merge(complete_df, ml_df, how='left', left_on=['Title', 'Year'], right_on=['movie_title', 'year'])
merged_disney_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,IMDBRating,imdbVotes,imdbID,total_gross,inflation_adjusted_gross,movieId,title,genres,movie_title,year
0,101 Dalmatians,1996,G,27 Nov 1996,103.0,Adventure|Comedy|Crime,Stephen Herek,Dodie Smith| John Hughes,Glenn Close| Jeff Daniels| Joely Richardson,English|Spanish,...,5.7,109762,tt0115433,153000000.0,1.386569e+09,1367.0,101 Dalmatians (1996),Adventure|Children|Comedy,101 Dalmatians,1996.0
1,101 Dalmatians,1996,G,27 Nov 1996,103.0,Adventure|Comedy|Crime,Stephen Herek,Dodie Smith| John Hughes,Glenn Close| Jeff Daniels| Joely Richardson,English|Spanish,...,5.7,109762,tt0115433,136189294.0,2.352021e+08,1367.0,101 Dalmatians (1996),Adventure|Children|Comedy,101 Dalmatians,1996.0
2,102 Dalmatians,2000,G,22 Nov 2000,100.0,Adventure|Comedy|Family,Kevin Lima,Dodie Smith| Kristen Buckley| Brian Regan,Glenn Close| Gérard Depardieu| Ioan Gruffudd,English,...,4.9,37045,tt0211181,66941559.0,1.053377e+08,3991.0,102 Dalmatians (2000),Children|Comedy,102 Dalmatians,2000.0
3,1492: Conquest of Paradise,1992,PG-13,09 Oct 1992,154.0,Adventure|Biography|Drama,Ridley Scott,Rose Bosch,Gérard Depardieu| Armand Assante| Sigourney We...,English|Spanish|Latin|Arabic,...,6.4,29990,tt0103594,7099531.0,1.371176e+07,8905.0,1492: Conquest of Paradise (1992),Adventure|Drama,1492: Conquest of Paradise,1992.0
4,"20,000 Leagues Under the Sea",1954,G,20 Jul 1955,127.0,Adventure|Drama|Family,Richard Fleischer,Earl Felton| Jules Verne,Kirk Douglas| James Mason| Paul Lukas,English,...,7.2,33152,tt0046672,28200000.0,2.840652e+08,1019.0,"20,000 Leagues Under the Sea (1954)",Adventure|Drama|Sci-Fi,"20,000 Leagues Under the Sea",1954.0


In [11]:
merged_disney_df.isna().sum()

Title                         0
Year                          0
Rated                         0
Released                      0
Runtime                       0
Genre                         0
Director                      1
Writer                        5
Actors                        0
Language                      0
Country                       0
Awards                      101
Metascore                     0
IMDBRating                    0
imdbVotes                     0
imdbID                        0
total_gross                   0
inflation_adjusted_gross      0
movieId                     165
title                       165
genres                      165
movie_title                 165
year                        165
dtype: int64

In [12]:
#genres na?
merged_disney_df.loc[merged_disney_df['genres'].isna()]

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,IMDBRating,imdbVotes,imdbID,total_gross,inflation_adjusted_gross,movieId,title,genres,movie_title,year
11,A Bug's Life,1998,G,25 Nov 1998,95.0,Animation|Adventure|Comedy,John Lasseter| Andrew Stanton,John Lasseter| Andrew Stanton| Joe Ranft,Kevin Spacey| Dave Foley| Julia Louis-Dreyfus,English,...,7.2,284819,tt0120623,162798565.0,270635135.0,NaN,NaN,NaN,NaN,NaN
12,A Civil Action,1998,PG-13,08 Jan 1999,115.0,Drama,Steven Zaillian,Jonathan Harr| Steven Zaillian,John Travolta| Robert Duvall| Kathleen Quinlan,English,...,6.6,28711,tt0120633,56709981.0,94274254.0,NaN,NaN,NaN,NaN,NaN
13,A Far Off Place,1993,PG,12 Mar 1993,108.0,Adventure|Drama|Family,Mikael Salomon,Lourens van der Post| Robert Caswell| Jonathan...,Reese Witherspoon| Ethan Embry| Jack Thompson,English,...,6.5,3556,tt0106868,12890752.0,24173059.0,NaN,NaN,NaN,NaN,NaN
14,A Goofy Movie,1995,G,07 Apr 1995,78.0,Animation|Adventure|Comedy,Kevin Lima,Jymn Magon| Chris Matheson| Brian Pimental,Bill Farmer| Jason Marsden| Jim Cummings,English,...,6.9,52129,tt0113198,35336439.0,62828837.0,NaN,NaN,NaN,NaN,NaN
15,The Gun in Betty Lou's Handbag,1992,PG-13,21 Aug 1992,89.0,Comedy|Mystery,Allan Moyle,Grace Cary Bickley,Penelope Ann Miller| Eric Thal| Alfre Woodard,English,...,5.1,1683,tt0104376,3591460.0,6936407.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,Red Rock West,1993,R,14 May 1993,98.0,Crime|Drama|Thriller,John Dahl,John Dahl| Rick Dahl,Nicolas Cage| Dennis Hopper| Lara Flynn Boyle,English,...,7.0,22020,tt0105226,2502551.0,4575683.0,NaN,NaN,NaN,NaN,NaN
612,Endurance,1998,G,14 May 1999,83.0,Biography|Drama|Sport,Leslie Woodhead| Bud Greenspan,NaN,Haile Gebrselassie| Shawananness Gebrselassie|...,English,...,6.3,419,tt0120659,229128.0,372669.0,NaN,NaN,NaN,NaN,NaN
614,Mars Needs Moms,2010,PG,11 Mar 2011,88.0,Animation|Action|Adventure,Simon Wells,Simon Wells| Wendy Wells| Berkeley Breathed,Seth Green| Joan Cusack| Dan Fogler,English,...,5.4,22234,tt1305591,21392758.0,25770522.0,NaN,NaN,NaN,NaN,NaN
615,The Jungle Book 2,2002,G,14 Feb 2003,72.0,Animation|Adventure|Comedy,Steve Trenbirth,Karl Geurs| Carter Crocker| Evan Spiliotopoulos,John Goodman| Haley Joel Osment| Tony Jay,English,...,5.4,16161,tt0283426,47901582.0,70542889.0,NaN,NaN,NaN,NaN,NaN


In [13]:
#genres na?
merged_disney_df.loc[merged_disney_df['genres'] == '(no genres listed)']

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,IMDBRating,imdbVotes,imdbID,total_gross,inflation_adjusted_gross,movieId,title,genres,movie_title,year
333,Pirates of the Caribbean: Dead Men Tell No Tales,2017,PG-13,26 May 2017,129.0,Action|Adventure|Fantasy,Joachim Rønning| Espen Sandberg,Jeff Nathanson| Terry Rossio| Ted Elliott,Johnny Depp| Geoffrey Rush| Javier Bardem,English|Spanish,...,6.5,292790,tt1790809,172558876.0,190756685.0,122896.0,Pirates of the Caribbean: Dead Men Tell No Tal...,(no genres listed),Pirates of the Caribbean: Dead Men Tell No Tales,2017.0


In [14]:
#Replace Genre "Not Rated" with "Not-Rated"

merged_disney_df['Rated'] = merged_disney_df['Rated'].str.replace('Not Rated','Not-Rated')

In [15]:
merged_disney_df.isna().sum()

Title                         0
Year                          0
Rated                         0
Released                      0
Runtime                       0
Genre                         0
Director                      1
Writer                        5
Actors                        0
Language                      0
Country                       0
Awards                      101
Metascore                     0
IMDBRating                    0
imdbVotes                     0
imdbID                        0
total_gross                   0
inflation_adjusted_gross      0
movieId                     165
title                       165
genres                      165
movie_title                 165
year                        165
dtype: int64

In [16]:
#movieId na?
merged_disney_df.loc[merged_disney_df['movieId'].isna()]

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,IMDBRating,imdbVotes,imdbID,total_gross,inflation_adjusted_gross,movieId,title,genres,movie_title,year
11,A Bug's Life,1998,G,25 Nov 1998,95.0,Animation|Adventure|Comedy,John Lasseter| Andrew Stanton,John Lasseter| Andrew Stanton| Joe Ranft,Kevin Spacey| Dave Foley| Julia Louis-Dreyfus,English,...,7.2,284819,tt0120623,162798565.0,270635135.0,NaN,NaN,NaN,NaN,NaN
12,A Civil Action,1998,PG-13,08 Jan 1999,115.0,Drama,Steven Zaillian,Jonathan Harr| Steven Zaillian,John Travolta| Robert Duvall| Kathleen Quinlan,English,...,6.6,28711,tt0120633,56709981.0,94274254.0,NaN,NaN,NaN,NaN,NaN
13,A Far Off Place,1993,PG,12 Mar 1993,108.0,Adventure|Drama|Family,Mikael Salomon,Lourens van der Post| Robert Caswell| Jonathan...,Reese Witherspoon| Ethan Embry| Jack Thompson,English,...,6.5,3556,tt0106868,12890752.0,24173059.0,NaN,NaN,NaN,NaN,NaN
14,A Goofy Movie,1995,G,07 Apr 1995,78.0,Animation|Adventure|Comedy,Kevin Lima,Jymn Magon| Chris Matheson| Brian Pimental,Bill Farmer| Jason Marsden| Jim Cummings,English,...,6.9,52129,tt0113198,35336439.0,62828837.0,NaN,NaN,NaN,NaN,NaN
15,The Gun in Betty Lou's Handbag,1992,PG-13,21 Aug 1992,89.0,Comedy|Mystery,Allan Moyle,Grace Cary Bickley,Penelope Ann Miller| Eric Thal| Alfre Woodard,English,...,5.1,1683,tt0104376,3591460.0,6936407.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,Red Rock West,1993,R,14 May 1993,98.0,Crime|Drama|Thriller,John Dahl,John Dahl| Rick Dahl,Nicolas Cage| Dennis Hopper| Lara Flynn Boyle,English,...,7.0,22020,tt0105226,2502551.0,4575683.0,NaN,NaN,NaN,NaN,NaN
612,Endurance,1998,G,14 May 1999,83.0,Biography|Drama|Sport,Leslie Woodhead| Bud Greenspan,NaN,Haile Gebrselassie| Shawananness Gebrselassie|...,English,...,6.3,419,tt0120659,229128.0,372669.0,NaN,NaN,NaN,NaN,NaN
614,Mars Needs Moms,2010,PG,11 Mar 2011,88.0,Animation|Action|Adventure,Simon Wells,Simon Wells| Wendy Wells| Berkeley Breathed,Seth Green| Joan Cusack| Dan Fogler,English,...,5.4,22234,tt1305591,21392758.0,25770522.0,NaN,NaN,NaN,NaN,NaN
615,The Jungle Book 2,2002,G,14 Feb 2003,72.0,Animation|Adventure|Comedy,Steve Trenbirth,Karl Geurs| Carter Crocker| Evan Spiliotopoulos,John Goodman| Haley Joel Osment| Tony Jay,English,...,5.4,16161,tt0283426,47901582.0,70542889.0,NaN,NaN,NaN,NaN,NaN


In [17]:
#merged_disney_df['title'] = merged_disney_df['title'].astype(str)

#So after merged, we need to find titles in IMDB lists without it's corresponding MovieLens movieID
#then we take the MovieLens movieId & genres to add into merged_disney_df

for i in merged_disney_df.index:
    if str(merged_disney_df['movieId'][i]) == 'nan':  
        for j in ml_df.index:
            if merged_disney_df['Title'][i] == ml_df['movie_title'][j]:
                print("--------------------------")
                print(merged_disney_df['Title'][i])
                print(ml_df['movie_title'][j])
                merged_disney_df['movieId'][i] = ml_df['movieId'][j]
                merged_disney_df['genres'][i] = ml_df['genres'][j]
                print("--------------------------")

--------------------------
Amy
Amy
--------------------------


/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


--------------------------
Amy
Amy
--------------------------
--------------------------
Black Panther
Black Panther
--------------------------
--------------------------
Black Widow
Black Widow
--------------------------
--------------------------
Black Widow
Black Widow
--------------------------
--------------------------
Captain Marvel
Captain Marvel
--------------------------
--------------------------
A Christmas Carol
A Christmas Carol
--------------------------
--------------------------
A Christmas Carol
A Christmas Carol
--------------------------
--------------------------
Earth
Earth
--------------------------
--------------------------
Earth
Earth
--------------------------
--------------------------
Earth
Earth
--------------------------
--------------------------
Houseguest
Houseguest
--------------------------
--------------------------
I'll Be Home for Christmas
I'll Be Home for Christmas
--------------------------
--------------------------
Night Crossing
Night Crossi

In [18]:
merged_disney_df.loc[merged_disney_df['Title'] == 'The Little Mermaid']

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,IMDBRating,imdbVotes,imdbID,total_gross,inflation_adjusted_gross,movieId,title,genres,movie_title,year
482,The Little Mermaid,1989,G,17 Nov 1989,83.0,Animation|Adventure|Comedy,Ron Clements| John Musker,John Musker| Ron Clements| Hans Christian Ande...,Jodi Benson| Samuel E. Wright| Rene Auberjonois,English|French,...,7.6,254101,tt0097757,111543479.0,243749487.0,193537.0,NaN,Animation|Fantasy,NaN,NaN


In [19]:
#We are going to add the Genre to genres column if it is nan or (no genres listed) because we will keep genres 
#column (from MovieLens) and drop Genre column from IMDB.
#The reason is MovieLens genres have longer genres list than IMDB

for i in merged_disney_df.index:
    if str(merged_disney_df['title'][i]) == 'nan' or str(merged_disney_df['genres'][i]) =='(no genres listed)':
        print("List of Movies that IMDB Genre nan will be replaced by MovieLens genres")
        print(merged_disney_df['Title'][i])
        print(merged_disney_df['Genre'][i])
        print(merged_disney_df['genres'][i])
        merged_disney_df['genres'][i] = merged_disney_df['Genre'][i]
        print("--------------------------")
        
    elif len(merged_disney_df['Genre'][i]) > len(merged_disney_df['genres'][i]):
        print(merged_disney_df['Genre'][i])
        print(merged_disney_df['genres'][i])
        merged_disney_df['genres'][i] = merged_disney_df['Genre'][i]
        print("--------------------------")

Adventure|Comedy|Family
Children|Comedy
--------------------------
Adventure|Biography|Drama
Adventure|Drama
--------------------------
Comedy|Drama|Family
Comedy
--------------------------
Comedy|Drama|Family
Comedy|Romance
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
A Bug's Life
Animation|Adventure|Comedy
nan
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
A Civil Action
Drama
nan
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
A Far Off Place
Adventure|Drama|Family
nan
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
A Goofy Movie
Animation|Adventure|Comedy
nan
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
The Gun in Betty Lou's Handbag
Comedy|Mystery
nan
--------------------------
List of Movies that IMDB Genre nan will be rep

/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()



Comedy
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
Million Dollar Arm
Biography|Drama|Sport
nan
--------------------------
Biography|Drama|History
Drama
--------------------------
Adventure|Sci-Fi|Thriller
Sci-Fi
--------------------------
Comedy|Crime|Drama
Comedy|Crime
--------------------------
Documentary|Sport
Documentary
--------------------------
Comedy|Fantasy|Horror
Comedy
--------------------------
Comedy|Family|Sci-Fi
Comedy|Sci-Fi
--------------------------
Action|Adventure|Comedy
Adventure|Sci-Fi
--------------------------
Comedy|Drama|Sport
Comedy|Drama
--------------------------
Action|Adventure|Mystery
Action|Adventure
--------------------------
Comedy|Drama|Romance
Comedy|Drama
--------------------------
Drama|Family|History
Children|Musical
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
Night Crossing
Drama|Family|History
Drama
--------------------------
Biograp

nan
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
The Rich Man's Wife
Thriller
nan
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
The Rock
Action|Adventure|Thriller
nan
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
The Rocketeer
Action|Adventure|Family
nan
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
The Rookie
Drama|Family|Sport
nan
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
The Royal Tenenbaums
Comedy|Drama
nan
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
The Santa Clause
Comedy|Drama|Family
nan
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
The Santa Clause 2
Comedy|Family|Fantasy
nan
--------------------------
List

In [20]:
#This first test list is ready for ML K-means Clustering

merged_disney_df.to_csv (r'output/ML_bb_ready.csv', index = None, header=True) 

### Load Completed IMDB TV show dataset

In [44]:
#Load the complete IMDB dataset
imdb_tv_df = pd.read_csv('input/IMDB_tv.csv')

imdb_tv_df['Rated'] = imdb_tv_df['Rated'].str.replace('Not Rated','Not-Rated')
imdb_tv_df['Title'] = imdb_tv_df['Title'].str.replace('.', '')

imdb_tv_df['Year'] = imdb_tv_df['Year'].str.split('–', 1).str[0]
imdb_tv_df['Year'] = imdb_tv_df['Year'].astype(int)
imdb_tv_df['Genre'] = imdb_tv_df['Genre'].str.replace(',', '|')
imdb_tv_df['Genre'] = imdb_tv_df['Genre'].str.replace(' ', '')

imdb_tv_df['Director'] = imdb_tv_df['Director'].str.replace(',', '|')
#imdb_tv_df['Director'] = imdb_tv_df['Director'].str.replace(' ', '')

imdb_tv_df['Writer'] = imdb_tv_df['Writer'].str.replace(',', '|')
#imdb_tv_df['Writer'] = imdb_tv_df['Writer'].str.replace(' ', '')

imdb_tv_df['Actors'] = imdb_tv_df['Actors'].str.replace(',', '|')
#imdb_tv_df['Actors'] = imdb_tv_df['Actors'].str.replace(' ', '')

imdb_tv_df['Language'] = imdb_tv_df['Language'].str.replace(',', '|')
imdb_tv_df['Language'] = imdb_tv_df['Language'].str.replace(' ', '')

imdb_tv_df['Country'] = imdb_tv_df['Country'].str.replace(',', '|')
#imdb_tv_df['Country'] = imdb_tv_df['Country'].str.replace(' ', '')

imdb_tv_df['imdbVotes'] = imdb_tv_df['imdbVotes'].astype(str)
imdb_tv_df['imdbVotes'] = imdb_tv_df['imdbVotes'].str.replace(',', '')

imdb_tv_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,Country,Awards,Metascore,IMDBRating,imdbVotes,imdbID
0,Ice Age: A Mammoth Christmas,2011,TV-PG,24 Nov 2011,26,Animation|Short|Adventure|Comedy|Family|Fantasy,Karen Disher,Sam Harper| Mike Reiss,Ray Romano| John Leguizamo| Denis Leary| Queen...,English,USA,1 nomination.,-1.0,6.5,9671,tt2100546
1,The Queen Family Singalong,2021,TV-PG,04 Nov 2021,41,Music|Musical,Hamish Hamilton,Brian Strickland,Darren Criss| Adam Lambert| Jimmie Allen,English,United States,NaN,-1.0,5.9,86,tt15716286
2,The Beatles: Get Back,2021,PG-13,25 Nov 2021,-1,Documentary|Biography|Music,Peter Jackson,NaN,The Beatles| John Lennon| Paul McCartney,English,United Kingdom| New Zealand| United States,4 wins & 4 nominations,-1.0,9.1,18501,tt9735318
3,Becoming Cousteau,2021,PG-13,24 Nov 2021,94,Documentary|Adventure|Biography,Liz Garbus,Mark Monroe| Pax Wassermann,Vincent Cassel| Carol Burnett| Louis Malle,English|French,United States,Nominated for 1 BAFTA Film Award2 wins & 11 no...,74.0,7.1,834,tt10334438
4,Hawkeye,2021,TV-14,24 Nov 2021,60,Action|Adventure|Crime,NaN,NaN,Jeremy Renner| Hailee Steinfeld| Florence Pugh,English|AmericanSign,United States,8 nominations,-1.0,7.7,142549,tt10160804


In [45]:
#Merge the IMDB tv shows dataset and MovieLens dataset on movie_title & year ONLY
merged_tv_df =pd.merge(imdb_tv_df, ml_df, how='left', left_on=['Title', 'Year'], right_on=['movie_title', 'year'])
merged_tv_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,Awards,Metascore,IMDBRating,imdbVotes,imdbID,movieId,title,genres,movie_title,year
0,Ice Age: A Mammoth Christmas,2011,TV-PG,24 Nov 2011,26,Animation|Short|Adventure|Comedy|Family|Fantasy,Karen Disher,Sam Harper| Mike Reiss,Ray Romano| John Leguizamo| Denis Leary| Queen...,English,...,1 nomination.,-1.0,6.5,9671,tt2100546,117922.0,Ice Age: A Mammoth Christmas (2011),Adventure|Animation|Children,Ice Age: A Mammoth Christmas,2011.0
1,The Queen Family Singalong,2021,TV-PG,04 Nov 2021,41,Music|Musical,Hamish Hamilton,Brian Strickland,Darren Criss| Adam Lambert| Jimmie Allen,English,...,NaN,-1.0,5.9,86,tt15716286,NaN,NaN,NaN,NaN,NaN
2,The Beatles: Get Back,2021,PG-13,25 Nov 2021,-1,Documentary|Biography|Music,Peter Jackson,NaN,The Beatles| John Lennon| Paul McCartney,English,...,4 wins & 4 nominations,-1.0,9.1,18501,tt9735318,NaN,NaN,NaN,NaN,NaN
3,Becoming Cousteau,2021,PG-13,24 Nov 2021,94,Documentary|Adventure|Biography,Liz Garbus,Mark Monroe| Pax Wassermann,Vincent Cassel| Carol Burnett| Louis Malle,English|French,...,Nominated for 1 BAFTA Film Award2 wins & 11 no...,74.0,7.1,834,tt10334438,NaN,NaN,NaN,NaN,NaN
4,Hawkeye,2021,TV-14,24 Nov 2021,60,Action|Adventure|Crime,NaN,NaN,Jeremy Renner| Hailee Steinfeld| Florence Pugh,English|AmericanSign,...,8 nominations,-1.0,7.7,142549,tt10160804,NaN,NaN,NaN,NaN,NaN


In [46]:
#Replace Genre "Not Rated" with "Not-Rated"
merged_tv_df['Rated'] = merged_tv_df['Rated'].str.replace('Not Rated','Not-Rated')

In [47]:
merged_tv_df.isna().sum()

Title            0
Year             0
Rated          164
Released        13
Runtime          0
Genre            6
Director       232
Writer         199
Actors          35
Language        18
Country         10
Awards         404
Metascore        0
IMDBRating       0
imdbVotes        0
imdbID           0
movieId        496
title          496
genres         496
movie_title    496
year           496
dtype: int64

In [48]:
#genres na?
merged_tv_df.loc[merged_tv_df['genres'] == '(no genres listed)']

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,Awards,Metascore,IMDBRating,imdbVotes,imdbID,movieId,title,genres,movie_title,year
352,How to Play Football,1944,Approved,15 Sep 1944,7,Animation|Short|Comedy,Jack Kinney,Rex Cox| Harry Reeves,Billy Bletcher| Pinto Colvig,English,...,Nominated for 1 Oscar. 1 nomination total,-1.0,7.1,1199,tt0036933,136453.0,How to Play Football (1944),(no genres listed),How to Play Football,1944.0
415,A Ring of Endless Light,2002,TV-G,23 Aug 2002,88,Drama|Family|Romance,Greg Beeman,Madeleine L'Engle (novel)| Marita Giovanni (te...,Mischa Barton| Ryan Merriman| Jared Padalecki|...,English,...,2 wins & 1 nomination.,-1.0,6.0,1165,tt0282123,143426.0,A Ring of Endless Light (2002),(no genres listed),A Ring of Endless Light,2002.0
419,Almost Angels,1962,Not-Rated,21 Dec 1962,93,Comedy|Family|Music,Steve Previn,Vernon Harris| Robert A. Stemmle,Vincent Winter| Sean Scully| Peter Weck,English,...,NaN,-1.0,6.6,547,tt0055740,170058.0,Almost Angels (1962),(no genres listed),Almost Angels,1962.0
508,Going to the Mat,2004,G,19 Mar 2004,120,Drama|Family|Sport,Stuart Gillard,Chris Nolan (story)| Laurie Nolan (story)| Chr...,Andrew Lawrence| Khleo Thomas| Alessandra Torr...,English,...,1 win & 3 nominations.,-1.0,6.9,1305,tt0399104,143235.0,Going to the Mat (2004),(no genres listed),Going to the Mat,2004.0
567,Life Is Ruff,2005,TV-G,15 Jul 2005,84,Comedy|Family,Charles Haid,Eddie Guzelian,Kyle Massey| Kay Panabaker| Mitchel Musso| Car...,English,...,NaN,-1.0,4.9,1290,tt0450455,142572.0,Life Is Ruff (2005),(no genres listed),Life Is Ruff,2005.0
635,Quints,2000,TV-G,18 Aug 2000,83,Comedy|Drama|Family,Bill Corcoran,Matthew Weisman (story)| Matthew Weisman (tele...,Kimberly J. Brown| Daniel Roebuck| Elizabeth M...,English,...,2 nominations.,-1.0,5.7,2243,tt0245380,136371.0,Quints (2000),(no genres listed),Quints,2000.0
643,Right on Track,2003,TV-G,21 Mar 2003,89,Biography|Drama|Family,Duwayne Dunham,Sally Nemeth| Bruce Graham,Beverley Mitchell| Brie Larson| Jon Lindstrom|...,English,...,NaN,-1.0,6.3,1233,tt0323810,140461.0,Right on Track (2003),(no genres listed),Right on Track,2003.0
716,The Jungle Book: Mowgli's Story,1998,G,28 Sep 1998,77,Adventure|Family,Nick Marck,Rudyard Kipling (book)| Jose Rivera (teleplay)...,Brandon Baker| Sherman Howard| Clancy Brown| P...,English,...,NaN,-1.0,4.2,677,tt0144262,165965.0,The Jungle Book: Mowgli's Story (1998),(no genres listed),The Jungle Book: Mowgli's Story,1998.0
731,The Prince and the Pauper,1962,TV-PG,11 Mar 1962,93,Adventure,Don Chaffey,Mark Twain| Jack Whittingham,Guy Williams| Laurence Naismith| Donald Houston,English,...,NaN,-1.0,6.3,295,tt1836776,170056.0,The Prince and the Pauper (1962),(no genres listed),The Prince and the Pauper,1962.0
745,The Thirteenth Year,1999,TV-G,15 May 1999,95,Comedy|Family|Fantasy,Duwayne Dunham,Jenny Arata (story)| Robert L. Baird (teleplay...,Chez Starbuck| Justin Jon Ross| Courtnee Drape...,English,...,1 nomination.,-1.0,6.0,5072,tt0200208,136590.0,The Thirteenth Year (1999),(no genres listed),The Thirteenth Year,1999.0


In [49]:
#genres na?
merged_tv_df.loc[merged_tv_df['Genre'].isna()]

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,Awards,Metascore,IMDBRating,imdbVotes,imdbID,movieId,title,genres,movie_title,year
94,Built for Mars: The Perseverance Rover,2021,NaN,02 May 2021,88,NaN,Mark Davis,Mark Davis,Steve French| Moogega Cooper| Adam Steltzner,NaN,...,NaN,-1.0,-1.0,50,tt13932270,NaN,NaN,NaN,NaN,NaN
211,Disney Holiday Magic Quest,2020,NaN,04 Dec 2020,45,NaN,Art Spigel,NaN,Raven-Symoné| Milo Manheim| Kylee Russell,English,...,NaN,-1.0,6.2,32,tt13522634,NaN,NaN,NaN,NaN,NaN
223,American Party Animals,2016,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,...,NaN,-1.0,-1.0,-1,tt6925732,NaN,NaN,NaN,NaN,NaN
285,Destination World,2018,NaN,NaN,-1,NaN,NaN,NaN,Rick Robles,English,...,NaN,-1.0,-1.0,-1,tt14630178,NaN,NaN,NaN,NaN,NaN
311,The Story of the Animated Drawing,1955,NaN,01 Nov 1955,50,NaN,NaN,NaN,NaN,NaN,...,NaN,-1.0,-1.0,-1,tt4903486,NaN,NaN,NaN,NaN,NaN
647,Rocky Mountain Animal Rescue,2018,NaN,NaN,-1,NaN,NaN,NaN,NaN,English,...,NaN,-1.0,8.4,27,tt9395780,NaN,NaN,NaN,NaN,NaN


In [50]:
#So after merged, we need to find titles in IMDB lists without it's corresponding MovieLens movieID
#then we take the MovieLens movieId & genres to add into merged_tv_df

for i in merged_tv_df.index:
    if str(merged_tv_df['movieId'][i]) == 'nan':  
        for j in ml_df.index:
            if merged_tv_df['Title'][i] == ml_df['movie_title'][j]:
                print("--------------------------")
                print(merged_tv_df['Title'][i])
                print(ml_df['movie_title'][j])
                merged_tv_df['movieId'][i] = ml_df['movieId'][j]
                merged_tv_df['genres'][i] = ml_df['genres'][j]
                print("--------------------------")

--------------------------
Spin
Spin
--------------------------


/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


--------------------------
Spin
Spin
--------------------------
--------------------------
Spin
Spin
--------------------------
--------------------------
Spin
Spin
--------------------------
--------------------------
Under Wraps
Under Wraps
--------------------------
--------------------------
Turner & Hooch
Turner & Hooch
--------------------------
--------------------------
Mulan
Mulan
--------------------------
--------------------------
Mulan
Mulan
--------------------------
--------------------------
Black Beauty
Black Beauty
--------------------------
--------------------------
Black Beauty
Black Beauty
--------------------------
--------------------------
Black Beauty
Black Beauty
--------------------------
--------------------------
Once Upon a Time
Once Upon a Time
--------------------------
--------------------------
Once Upon a Time
Once Upon a Time
--------------------------
--------------------------
Once Upon a Time
Once Upon a Time
--------------------------
----------

In [60]:
merged_tv_df.loc[merged_tv_df['Title'] == 'Lifted']

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,Awards,Metascore,IMDBRating,imdbVotes,imdbID,movieId,title,genres,movie_title,year
570,Lifted,2006,G,29 Jun 2007,5,Animation|Short|Comedy,Gary Rydstrom,Gary Rydstrom| Jeff Pidgeon| Maxwell Brace IV,Sheb Wooley,None,...,Nominated for 1 Oscar. 1 nomination total,-1.0,7.8,19290,tt0945571,133335.0,NaN,Children|Drama,NaN,NaN


In [61]:
merged_tv_df.loc[merged_tv_df['Genre'].isna()]

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,Awards,Metascore,IMDBRating,imdbVotes,imdbID,movieId,title,genres,movie_title,year


In [62]:
#drop the column if Genre is Null

merged_tv_df.dropna(subset = ['Genre'], inplace=True)

In [63]:
merged_tv_df.loc[merged_tv_df['Genre'].isna()]

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,Awards,Metascore,IMDBRating,imdbVotes,imdbID,movieId,title,genres,movie_title,year


In [64]:
#We are going to add the Genre to genres column if it is nan or (no genres listed) because we will keep genres 
#column (from MovieLens) and drop Genre column from IMDB.
#The reason is MovieLens genres have longer genres list than IMDB

for i in merged_disney_df.index:
    if str(merged_disney_df['title'][i]) == 'nan' or str(merged_disney_df['genres'][i]) =='(no genres listed)':
        print("List of Movies that IMDB Genre nan will be replaced by MovieLens genres")
        print(merged_disney_df['Title'][i])
        print(merged_disney_df['Genre'][i])
        print(merged_disney_df['genres'][i])
        merged_disney_df['genres'][i] = merged_disney_df['Genre'][i]
        print("--------------------------")
        
    elif len(merged_disney_df['Genre'][i]) > len(merged_disney_df['genres'][i]):
        print(merged_disney_df['Genre'][i])
        print(merged_disney_df['genres'][i])
        merged_disney_df['genres'][i] = merged_disney_df['Genre'][i]
        print("--------------------------")  

List of Movies that IMDB Genre nan will be replaced by MovieLens genres
A Bug's Life
Animation|Adventure|Comedy
Animation|Adventure|Comedy
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
A Civil Action
Drama
Drama
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
A Far Off Place
Adventure|Drama|Family
Adventure|Drama|Family
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
A Goofy Movie
Animation|Adventure|Comedy
Animation|Adventure|Comedy
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
The Gun in Betty Lou's Handbag
Comedy|Mystery
Comedy|Mystery
--------------------------
List of Movies that IMDB Genre nan will be replaced by MovieLens genres
A Kid in King Arthur's Court
Adventure|Comedy|Family
Adventure|Comedy|Family
--------------------------
List of Movies that IMDB Genre nan will be r

/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [65]:
merged_tv_df.isna().sum()

Title            0
Year             0
Rated            0
Released        10
Runtime          0
Genre            0
Director       228
Writer         194
Actors          32
Language        15
Country          7
Awards         398
Metascore        0
IMDBRating       0
imdbVotes        0
imdbID           0
movieId        470
title          490
genres         470
movie_title    490
year           490
dtype: int64

In [66]:
#Make a decision to fill Rated NULL to "G"
merged_tv_df['Rated'].fillna('G', inplace = True)

In [67]:
merged_tv_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,...,Awards,Metascore,IMDBRating,imdbVotes,imdbID,movieId,title,genres,movie_title,year
0,Ice Age: A Mammoth Christmas,2011,TV-PG,24 Nov 2011,26,Animation|Short|Adventure|Comedy|Family|Fantasy,Karen Disher,Sam Harper| Mike Reiss,Ray Romano| John Leguizamo| Denis Leary| Queen...,English,...,1 nomination.,-1.0,6.5,9671,tt2100546,117922.0,Ice Age: A Mammoth Christmas (2011),Adventure|Animation|Children,Ice Age: A Mammoth Christmas,2011.0
1,The Queen Family Singalong,2021,TV-PG,04 Nov 2021,41,Music|Musical,Hamish Hamilton,Brian Strickland,Darren Criss| Adam Lambert| Jimmie Allen,English,...,NaN,-1.0,5.9,86,tt15716286,NaN,NaN,NaN,NaN,NaN
2,The Beatles: Get Back,2021,PG-13,25 Nov 2021,-1,Documentary|Biography|Music,Peter Jackson,NaN,The Beatles| John Lennon| Paul McCartney,English,...,4 wins & 4 nominations,-1.0,9.1,18501,tt9735318,NaN,NaN,NaN,NaN,NaN
3,Becoming Cousteau,2021,PG-13,24 Nov 2021,94,Documentary|Adventure|Biography,Liz Garbus,Mark Monroe| Pax Wassermann,Vincent Cassel| Carol Burnett| Louis Malle,English|French,...,Nominated for 1 BAFTA Film Award2 wins & 11 no...,74.0,7.1,834,tt10334438,NaN,NaN,NaN,NaN,NaN
4,Hawkeye,2021,TV-14,24 Nov 2021,60,Action|Adventure|Crime,NaN,NaN,Jeremy Renner| Hailee Steinfeld| Florence Pugh,English|AmericanSign,...,8 nominations,-1.0,7.7,142549,tt10160804,NaN,NaN,NaN,NaN,NaN


In [68]:
#This first test list is ready for ML K-means Clustering

merged_tv_df.to_csv (r'output/ML_tv_ready.csv', index = None, header=True) 

In [37]:
# #make a running number for column test
# ml_df['test'] = np.arange(len(ml_df))

# #add constant string in front of the running number for Primary Key
# ml_df['test'] = 'wc-00000' + ml_df['test'].astype(str)

# ml_df.head()


In [38]:
# #check some titles
# merged_disney_df[merged_disney_df.Title.str.startswith('WAL')]

In [39]:
# #check some titles
# ml_df[ml_df.title.str.startswith('WAL')]

In [40]:
# #check some titles
# ml_df[ml_df.title.str.startswith('Déjà')]

### Load MovieLens Ratings dataset

In [41]:
# #load Movie Lens ratings dataset
# ml_ratings_df = pd.read_csv('input/ratings.csv')
# ml_ratings_df.head()

In [42]:
# ml_ratings_df.loc[ml_ratings_df['movieId'] == 49278]

In [43]:
# ml_ratings_df[ml_ratings_df.movieId.str.startswith('49278')]